# 🎯 Identificación de Parámetros - Principio de Mínima Acción

## Formulación Lagrangiana del Levitador Magnético + 9 Metaheurísticos

**Lagrangiano:** $\mathcal{L} = \frac{1}{2}m\dot{y}^2 + mgy + \frac{1}{2}L(y)i^2$

**Parámetros a identificar (5D):** $k_0, k, a, m, R$

**Metaheurísticos implementados:**
1. Differential Evolution (DE/best/1)
2. Particle Swarm Optimization (PSO)
3. Grey Wolf Optimizer (GWO)
4. Whale Optimization Algorithm (WOA)
5. Sine-Cosine Algorithm (SCA)
6. Salp Swarm Algorithm (SSA)
7. Harris Hawks Optimization (HHO)
8. Arithmetic Optimization Algorithm (AOA)
9. Covariance Matrix Adaptation (CMA-ES)
---

In [ ]:
# Setup
import subprocess, sys, platform
print("🔍 Verificando GPU...")
try:
    print(subprocess.check_output(['nvidia-smi'], text=True))
except: print("⚠️ Sin GPU")

IN_COLAB = 'google.colab' in sys.modules
!pip install -q "numpy<2.1.0" "ml-dtypes>=0.4.0,<0.5.0"
if IN_COLAB:
    !pip install -q "jax[cuda12]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
else:
    !pip install -q jax jaxlib
!pip install -q scipy pandas matplotlib tqdm plotly ipywidgets
print("✅ Dependencias instaladas")

In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit, vmap
from jax.lax import scan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json, time, os
from datetime import datetime
from tqdm.auto import tqdm
from functools import partial
import plotly.graph_objects as go
from IPython.display import display, update_display

print(f"JAX devices: {jax.devices()}")
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
REPO_URL = "https://github.com/JRavenelco/levitador-benchmark.git"
if not os.path.exists("levitador-benchmark"):
    !git clone {REPO_URL}
    %cd levitador-benchmark
elif os.getcwd().split(os.sep)[-1] != "levitador-benchmark":
    %cd levitador-benchmark
!ls -la data/*.txt 2>/dev/null || echo "No data"
!ls -la data/sesiones_kan_pinn/*.txt 2>/dev/null || echo "No KAN-PINN data" 

In [ ]:
# Archivos disponibles para identificacion
DATA_FILES = {
    'MONIT_SWEEP': 'data/MONIT_SWEEP.txt',           # MEJOR: 11 setpoints, 5-18mm
    'datos_zonas_faltantes': 'data/datos_zonas_faltantes.txt',  # 10 setpoints, 9-18mm
    'MONIT_MAESTRO': 'data/MONIT_MAESTRO.txt',       # 5 setpoints, 4.5-10mm
    'datos_levitador': 'data/datos_levitador.txt',   # Original (solo 2 setpoints)
}

def load_data(filepath, subsample=1):
    df = pd.read_csv(filepath, sep=r'\s+', comment='#', header=None)
    if subsample > 1: df = df.iloc[::subsample].reset_index(drop=True)
    if df.shape[1] >= 6:
        return {'t': df.iloc[:,0].values.astype(np.float32),
                'sp': df.iloc[:,1].values.astype(np.float32),
                'y': df.iloc[:,2].values.astype(np.float32),
                'dy': df.iloc[:,3].values.astype(np.float32),
                'i': df.iloc[:,4].values.astype(np.float32),
                'u': df.iloc[:,5].values.astype(np.float32)}
    else:
        return {'t': df.iloc[:,0].values.astype(np.float32),
                'sp': df.iloc[:,1].values.astype(np.float32),
                'y': df.iloc[:,1].values.astype(np.float32),
                'i': df.iloc[:,2].values.astype(np.float32),
                'u': df.iloc[:,3].values.astype(np.float32)}

def analyze_data(data, name):
    sp_range = (data['sp'].max() - data['sp'].min()) * 1000
    y_range = (data['y'].max() - data['y'].min()) * 1000
    i_range = data['i'].max() - data['i'].min()
    n_setpoints = len(np.unique(np.round(data['sp'], 5)))
    print(f"Dataset {name}: {len(data['t'])} muestras, {n_setpoints} setpoints")
    print(f"   SP: {data['sp'].min()*1000:.1f}-{data['sp'].max()*1000:.1f}mm | y: {y_range:.1f}mm | i: {i_range:.2f}A")
    return {'name': name, 'samples': len(data['t']), 'setpoints': n_setpoints, 
            'sp_range': sp_range, 'y_range': y_range, 'i_range': i_range}

print("="*60)
print("ARCHIVOS DISPONIBLES PARA IDENTIFICACION")
print("="*60)
for name, path in DATA_FILES.items():
    try:
        d = load_data(path, 1)
        analyze_data(d, name)
    except Exception as e:
        print(f"Error {name}: {e}")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════
# 🎯 SELECCIÓN DE DATOS - Modificar aquí
# ═══════════════════════════════════════════════════════════
SELECTED_FILES = ['MONIT_SWEEP']  # Opciones: 'MONIT_SWEEP', 'datos_zonas_faltantes', 'MONIT_MAESTRO', 'datos_levitador'
COMBINE_DATA = False  # True para combinar múltiples archivos
SUBSAMPLE = 5  # Submuestreo para acelerar (1 = sin submuestreo)

# Cargar datos seleccionados
if COMBINE_DATA and len(SELECTED_FILES) > 1:
    print(f"🔗 Combinando: {SELECTED_FILES}")
    datasets = [load_data(DATA_FILES[f], SUBSAMPLE) for f in SELECTED_FILES]
    # Concatenar con offset de tiempo
    t_offset = 0
    combined = {k: [] for k in ['t', 'sp', 'y', 'i', 'u']}
    for ds in datasets:
        combined['t'].extend(ds['t'] + t_offset)
        for k in ['sp', 'y', 'i', 'u']:
            combined[k].extend(ds[k])
        t_offset = combined['t'][-1] + 0.01
    data = {k: np.array(v, dtype=np.float32) for k, v in combined.items()}
    DATA_NAME = '+'.join(SELECTED_FILES)
else:
    DATA_NAME = SELECTED_FILES[0]
    data = load_data(DATA_FILES[DATA_NAME], SUBSAMPLE)

stats = analyze_data(data, DATA_NAME)
print(f"\n✅ Dataset cargado: {DATA_NAME}")
print(f"   Calidad para identificación: {'⭐⭐⭐ EXCELENTE' if stats['setpoints'] >= 8 else '⭐⭐ BUENA' if stats['setpoints'] >= 4 else '⭐ BÁSICA'}")

# Visualización
fig, ax = plt.subplots(3, 1, figsize=(14, 7), sharex=True)
ax[0].plot(data['t'], data['sp']*1000, 'g--', lw=1, alpha=0.7, label='Setpoint')
ax[0].plot(data['t'], data['y']*1000, 'b-', lw=1, label='Posición')
ax[0].set_ylabel('y [mm]'); ax[0].legend(); ax[0].grid(True)
ax[0].set_title(f'Dataset: {DATA_NAME} ({stats["samples"]} muestras, {stats["setpoints"]} setpoints)', fontweight='bold')
ax[1].plot(data['t'], data['i'], 'r-', lw=1); ax[1].set_ylabel('i [A]'); ax[1].grid(True)
ax[2].plot(data['t'], data['u'], 'purple', lw=1); ax[2].set_ylabel('u [V]'); ax[2].set_xlabel('t [s]'); ax[2].grid(True)
plt.tight_layout(); plt.show()

In [ ]:
t_jax = jnp.array(data['t'])
y_jax = jnp.array(data['y'])
i_jax = jnp.array(data['i'])
u_jax = jnp.array(data['u'])

dt_jax = jnp.diff(t_jax, prepend=t_jax[0])
dt_jax = dt_jax.at[0].set(dt_jax[1])
dt_mean = float(jnp.mean(dt_jax))

dy_jax = jnp.gradient(y_jax, dt_mean)
ddy_jax = jnp.gradient(dy_jax, dt_mean)
di_jax = jnp.gradient(i_jax, dt_mean)

y0, i0 = float(y_jax[0]), float(i_jax[0])
print(f"✅ GPU: dt={dt_mean:.4f}s, y0={y0*1000:.2f}mm, i0={i0:.3f}A")

## Modelo Físico Lagrangiano
$$L(y) = k_0 + \frac{k}{1+y/a}, \quad \frac{dL}{dy} = -\frac{k}{a(1+y/a)^2}$$
**Residuos E-L:** $R_{mec} = m\ddot{y} - \frac{1}{2}i^2\frac{dL}{dy} - mg$, $R_{elec} = L\dot{i} + \frac{dL}{dy}\dot{y}i + Ri - u$

In [ ]:
GRAVITY = 9.81
M_REF, R_REF, K0_REF, A_REF = 0.009, 2.2, 0.0035, 0.0052
print(f"Ref: m={M_REF*1000}g, R={R_REF}ohm, k0={K0_REF*1000}mH, a={A_REF*1000}mm")

@jit
def dynamics(state, u, params, g):
    # Calcula derivadas del sistema: [dy/dt, d2y/dt2, di/dt]
    k0, k, a, m, R = params
    y, dy, i = state
    y = jnp.clip(y, 1e-6, 0.03)  # Evitar singularidades
    i = jnp.clip(i, 0, 5)
    
    denom = 1.0 + y/a
    L = k0 + k/denom
    dL = -k/(a * denom**2)
    
    # Fuerza magnetica: F_mag = -0.5 * i^2 * dL/dy (atractiva, hacia arriba cuando dL<0)
    F_mag = -0.5 * i * i * dL
    ddy = (F_mag - m*g) / m
    
    # Ecuacion electrica: L*di/dt + dL/dy*dy/dt*i + R*i = u
    di = (u - R*i - dL*dy*i) / L
    
    return jnp.array([dy, ddy, di])

@jit
def simulate_trajectory_rk4(params, u_data, dt_data, y0, i0, g):
    # Integrador RK4 - Estable para ODEs rigidas
    state = jnp.array([y0, 0.0, i0])
    
    def rk4_step(state, inputs):
        u, dt = inputs
        k1 = dynamics(state, u, params, g)
        k2 = dynamics(state + 0.5*dt*k1, u, params, g)
        k3 = dynamics(state + 0.5*dt*k2, u, params, g)
        k4 = dynamics(state + dt*k3, u, params, g)
        new_state = state + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
        # Aplicar limites fisicos
        new_state = new_state.at[0].set(jnp.clip(new_state[0], 0, 0.03))  # y
        new_state = new_state.at[2].set(jnp.clip(new_state[2], 0, 5))     # i
        return new_state, jnp.array([new_state[0], new_state[2], new_state[1]])
    
    _, out = scan(rk4_step, state, (u_data, dt_data))
    return out[:,0], out[:,1], out[:,2]  # y, i, dy

# Usar RK4 como integrador principal
simulate_trajectory = simulate_trajectory_rk4

@jit 
def euler_lagrange_residuals(y, dy, ddy, i, di, u, params):
    k0, k, a, m, R = params
    y = jnp.clip(y, 1e-6, 0.03)
    denom = 1.0 + y/a
    L = k0 + k/denom
    dL = -k/(a * denom**2)
    R_mec = m*ddy + 0.5*i**2*dL + m*GRAVITY  # Corregido: +0.5 porque F_mag = -0.5*i^2*dL
    R_elec = L*di + dL*dy*i + R*i - u
    return R_mec, R_elec

euler_lagrange_vec = jit(vmap(euler_lagrange_residuals, in_axes=(0,0,0,0,0,0,None)))

# Validar con parametros de referencia
print("Validando modelo con parametros de referencia...")
y_test, i_test, dy_test = simulate_trajectory(jnp.array([K0_REF, K0_REF, A_REF, M_REF, R_REF]), u_jax, dt_jax, y0, i0, GRAVITY)
print(f"  y: min={float(jnp.min(y_test))*1000:.2f}mm, max={float(jnp.max(y_test))*1000:.2f}mm")
print(f"  i: min={float(jnp.min(i_test)):.3f}A, max={float(jnp.max(i_test)):.3f}A")
print("✅ Modelo Lagrangiano con RK4 definido")

In [ ]:
W_POS, W_CUR, W_EL, W_EN = 10000.0, 1.0, 100.0, 10.0
print(f"Pesos: W_pos={W_POS}, W_cur={W_CUR}, W_EL={W_EL}, W_energy={W_EN}")

@jit
def fitness_physics(params, u_data, dt_data, y_data, i_data, dy_data, ddy_data, di_data, y0, i0, g, dt):
    y_sim, i_sim, dy_sim = simulate_trajectory(params, u_data, dt_data, y0, i0, g)
    mse_y = jnp.mean((y_data - y_sim)**2)
    mse_i = jnp.mean((i_data - i_sim)**2)
    
    ddy_sim = jnp.gradient(dy_sim, dt)
    di_sim = jnp.gradient(i_sim, dt)
    R_mec, R_elec = euler_lagrange_vec(y_sim, dy_sim, ddy_sim, i_sim, di_sim, u_data, params)
    mse_el = jnp.mean(R_mec**2)/(params[3]**2) + jnp.mean(R_elec**2)
    
    k0, k, a, m, R = params
    L_ind = k0 + k/(1.0 + y_sim/a)
    E = 0.5*m*dy_sim**2 - m*g*y_sim + 0.5*L_ind*i_sim**2
    dE = jnp.gradient(E, dt)
    R_en = dE - (u_data*i_sim - R*i_sim**2)
    mse_en = jnp.mean(R_en**2)
    
    return jnp.nan_to_num(W_POS*mse_y + W_CUR*mse_i + W_EL*mse_el + W_EN*mse_en, nan=1e6)

fitness_pop = jit(vmap(fitness_physics, in_axes=(0,None,None,None,None,None,None,None,None,None,None,None)))
print("✅ Fitness Physics-Informed definido")

## 🧬 9 Algoritmos Metaheurísticos con Visualización 3D en Tiempo Real
Cada optimizador muestra la evolución de la población en el espacio (k0, k, a)

In [ ]:
# ═══════════════════════════════════════════════════════════
# 🎯 OPTIMIZADOR CON VISUALIZACIÓN 3D EN TIEMPO REAL (PLOTLY)
# ═══════════════════════════════════════════════════════════

def run_optimizer_interactive(name, optimizer_step_fn, fitness_fn, bounds, pop_size=100, max_iter=200, seed=42, ref=None):
    # Ejecuta optimizador con visualizacion 3D interactiva
    key = random.PRNGKey(seed)
    n = bounds.shape[0]
    ranges = bounds[:,1] - bounds[:,0]
    
    # Inicializar población
    key, sk = random.split(key)
    pop = random.uniform(sk, (pop_size, n)) * ranges + bounds[:,0]
    if ref is not None:
        key, sk = random.split(key)
        seeded = ref + random.normal(sk, (pop_size//4, n))*0.1*ranges
        pop = pop.at[:pop_size//4].set(jnp.clip(seeded, bounds[:,0], bounds[:,1]))
    
    fit = fitness_fn(pop)
    best_idx = jnp.argmin(fit)
    best, best_fit = pop[best_idx], fit[best_idx]
    hist = {'best': [], 'pop_history': []}
    
    # Crear figura 3D interactiva
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=np.array(pop[:,0])*1000, y=np.array(pop[:,1])*1000, z=np.array(pop[:,2])*1000,
        mode='markers', marker=dict(size=4, color=np.log10(np.array(fit)+1e-10), colorscale='Viridis', showscale=True),
        name='Población'))
    fig.add_trace(go.Scatter3d(
        x=[K0_REF*1000], y=[K0_REF*1000], z=[A_REF*1000],
        mode='markers', marker=dict(size=10, color='red', symbol='diamond'), name='Referencia'))
    fig.update_layout(
        template="plotly_dark",
        title=f"{name} - Gen 0 | Fitness: {float(best_fit):.2e}",
        scene=dict(
            xaxis=dict(title='k0 [mH]', range=[bounds[0,0]*1000, bounds[0,1]*1000]),
            yaxis=dict(title='k [mH]', range=[bounds[1,0]*1000, bounds[1,1]*1000]),
            zaxis=dict(title='a [mm]', range=[bounds[2,0]*1000, bounds[2,1]*1000])),
        width=800, height=600)
    
    display_id = f"opt_{name}_{int(time.time())}"
    display(fig, display_id=display_id)
    
    # Bucle de optimización
    for gen in range(max_iter):
        key, sk = random.split(key)
        pop, fit, best, best_fit = optimizer_step_fn(pop, fit, best, best_fit, bounds, sk, gen, max_iter)
        hist['best'].append(float(best_fit))
        
        # Actualizar visualización cada 5 generaciones
        if gen % 5 == 0:
            with fig.batch_update():
                fig.data[0].x = np.array(pop[:,0])*1000
                fig.data[0].y = np.array(pop[:,1])*1000
                fig.data[0].z = np.array(pop[:,2])*1000
                fig.data[0].marker.color = np.log10(np.array(fit)+1e-10)
                fig.layout.title.text = f"{name} - Gen {gen} | Fitness: {float(best_fit):.2e} | m={float(best[3])*1000:.1f}g"
            update_display(fig, display_id=display_id)
    
    # Actualización final
    with fig.batch_update():
        fig.layout.title.text = f"✅ {name} COMPLETO | Fitness: {float(best_fit):.2e}"
    update_display(fig, display_id=display_id)
    
    return best, best_fit, hist

print("✅ Framework de optimización interactiva definido")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 🧬 DEFINICIÓN DE 9 METAHEURÍSTICOS
# ═══════════════════════════════════════════════════════════

# 1. Differential Evolution (DE/best/1)
@partial(jit, static_argnums=(6,7))
def de_step(pop, fit, best, best_fit, bounds, key, gen, max_iter, F=0.7, CR=0.85):
    n = pop.shape[1]
    k1, k2, k3 = random.split(key, 3)
    a_idx = random.randint(k1, (len(pop),), 0, len(pop))
    b_idx = random.randint(k2, (len(pop),), 0, len(pop))
    mut = best + F*(pop[a_idx] - pop[b_idx])
    trial = jnp.where(random.uniform(k3, pop.shape) < CR, mut, pop)
    trial = jnp.clip(trial, bounds[:,0], bounds[:,1])
    trial_fit = fitness_fn(trial)
    improved = trial_fit < fit
    pop = jnp.where(improved[:,None], trial, pop)
    fit = jnp.where(improved, trial_fit, fit)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 2. Particle Swarm Optimization (PSO)
def pso_init(pop, bounds, key):
    vel = random.uniform(key, pop.shape, minval=-0.1, maxval=0.1) * (bounds[:,1] - bounds[:,0])
    return {'vel': vel, 'pbest': pop.copy(), 'pbest_fit': jnp.full(len(pop), jnp.inf)}

pso_state = None
@partial(jit, static_argnums=(6,7))
def pso_step(pop, fit, best, best_fit, bounds, key, gen, max_iter, w=0.7, c1=1.5, c2=1.5):
    global pso_state
    if pso_state is None or gen == 0:
        pso_state = pso_init(pop, bounds, key)
    vel, pbest, pbest_fit = pso_state['vel'], pso_state['pbest'], pso_state['pbest_fit']
    k1, k2 = random.split(key)
    r1, r2 = random.uniform(k1, pop.shape), random.uniform(k2, pop.shape)
    vel = w*vel + c1*r1*(pbest - pop) + c2*r2*(best - pop)
    pop = jnp.clip(pop + vel, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    improved = fit < pbest_fit
    pbest = jnp.where(improved[:,None], pop, pbest)
    pbest_fit = jnp.where(improved, fit, pbest_fit)
    pso_state = {'vel': vel, 'pbest': pbest, 'pbest_fit': pbest_fit}
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 3. Grey Wolf Optimizer (GWO)
@partial(jit, static_argnums=(6,7))
def gwo_step(pop, fit, best, best_fit, bounds, key, gen, max_iter):
    sorted_idx = jnp.argsort(fit)
    alpha, beta, delta = pop[sorted_idx[0]], pop[sorted_idx[1]], pop[sorted_idx[2]]
    a = 2.0 * (1.0 - gen/max_iter)
    k1, k2, k3 = random.split(key, 3)
    r1, r2 = random.uniform(k1, pop.shape), random.uniform(k2, pop.shape)
    A1, C1 = 2*a*r1 - a, 2*r2
    D_alpha = jnp.abs(C1*alpha - pop)
    X1 = alpha - A1*D_alpha
    r1, r2 = random.uniform(k2, pop.shape), random.uniform(k3, pop.shape)
    A2, C2 = 2*a*r1 - a, 2*r2
    X2 = beta - A2*jnp.abs(C2*beta - pop)
    r1, r2 = random.uniform(k3, pop.shape), random.uniform(k1, pop.shape)
    A3, C3 = 2*a*r1 - a, 2*r2
    X3 = delta - A3*jnp.abs(C3*delta - pop)
    pop = jnp.clip((X1 + X2 + X3)/3, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 4. Whale Optimization Algorithm (WOA)
@partial(jit, static_argnums=(6,7))
def woa_step(pop, fit, best, best_fit, bounds, key, gen, max_iter):
    a = 2.0 * (1.0 - gen/max_iter)
    k1, k2, k3 = random.split(key, 3)
    r, l, p = random.uniform(k1, pop.shape), random.uniform(k2, pop.shape)*2 - 1, random.uniform(k3, (len(pop),1))
    A, C = 2*a*r - a, 2*random.uniform(k2, pop.shape)
    b = 1.0
    D = jnp.abs(best - pop)
    spiral = D * jnp.exp(b*l) * jnp.cos(2*jnp.pi*l) + best
    encircle = best - A*jnp.abs(C*best - pop)
    pop = jnp.where(p < 0.5, encircle, spiral)
    pop = jnp.clip(pop, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 5. Sine-Cosine Algorithm (SCA)
@partial(jit, static_argnums=(6,7))
def sca_step(pop, fit, best, best_fit, bounds, key, gen, max_iter):
    a = 2.0 * (1.0 - gen/max_iter)
    k1, k2, k3, k4 = random.split(key, 4)
    r1 = a * random.uniform(k1, pop.shape)
    r2 = 2*jnp.pi*random.uniform(k2, pop.shape)
    r3 = 2*random.uniform(k3, pop.shape)
    r4 = random.uniform(k4, (len(pop),1))
    update_sin = pop + r1*jnp.sin(r2)*jnp.abs(r3*best - pop)
    update_cos = pop + r1*jnp.cos(r2)*jnp.abs(r3*best - pop)
    pop = jnp.where(r4 < 0.5, update_sin, update_cos)
    pop = jnp.clip(pop, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 6. Salp Swarm Algorithm (SSA)
@partial(jit, static_argnums=(6,7))
def ssa_step(pop, fit, best, best_fit, bounds, key, gen, max_iter):
    c1 = 2 * jnp.exp(-((4*gen/max_iter)**2))
    k1, k2, k3 = random.split(key, 3)
    c2, c3 = random.uniform(k1, pop.shape), random.uniform(k2, pop.shape)
    ranges = bounds[:,1] - bounds[:,0]
    leader_update = best + c1*((ranges)*c2 + bounds[:,0]) * jnp.where(c3 < 0.5, 1, -1)
    n_leaders = len(pop)//2
    new_pop = pop.at[:n_leaders].set(jnp.clip(leader_update[:n_leaders], bounds[:,0], bounds[:,1]))
    for i in range(n_leaders, len(pop)):
        new_pop = new_pop.at[i].set((new_pop[i] + new_pop[i-1])/2)
    pop = jnp.clip(new_pop, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 7. Harris Hawks Optimization (HHO)
@partial(jit, static_argnums=(6,7))
def hho_step(pop, fit, best, best_fit, bounds, key, gen, max_iter):
    E0 = 2*random.uniform(key, (len(pop),1)) - 1
    E = 2*E0*(1 - gen/max_iter)
    k1, k2, k3 = random.split(key, 3)
    q, r = random.uniform(k1, (len(pop),1)), random.uniform(k2, pop.shape)
    Xm = jnp.mean(pop, axis=0)
    rand_idx = random.randint(k3, (len(pop),), 0, len(pop))
    explore = pop[rand_idx] - r*jnp.abs(pop[rand_idx] - 2*r*pop)
    exploit = best - E*jnp.abs(best - pop)
    pop = jnp.where(jnp.abs(E) >= 1, explore, exploit)
    pop = jnp.clip(pop, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 8. Arithmetic Optimization Algorithm (AOA)
@partial(jit, static_argnums=(6,7))
def aoa_step(pop, fit, best, best_fit, bounds, key, gen, max_iter):
    MOA = 0.2 + (gen/max_iter)*0.8
    MOP = 1 - ((gen/max_iter)**(1/5))
    k1, k2, k3 = random.split(key, 3)
    r1, r2, r3 = random.uniform(k1, pop.shape), random.uniform(k2, (len(pop),1)), random.uniform(k3, pop.shape)
    ranges = bounds[:,1] - bounds[:,0]
    mul = best / (MOP + 1e-10) * ((ranges)*r1 + bounds[:,0])
    div = best * MOP * ((ranges)*r1 + bounds[:,0])
    add = best - MOP * ((ranges)*r1 + bounds[:,0])
    sub = best + MOP * ((ranges)*r1 + bounds[:,0])
    arith = jnp.where(r3 < 0.5, mul, div)
    update = jnp.where(r3 < 0.5, add, sub)
    pop = jnp.where(r2 < MOA, arith, update)
    pop = jnp.clip(pop, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

# 9. Simplified CMA-ES
@partial(jit, static_argnums=(6,7))
def cmaes_step(pop, fit, best, best_fit, bounds, key, gen, max_iter):
    mu = len(pop)//4
    sorted_idx = jnp.argsort(fit)[:mu]
    elite = pop[sorted_idx]
    mean = jnp.mean(elite, axis=0)
    sigma = 0.3 * (1 - gen/max_iter) * (bounds[:,1] - bounds[:,0])
    new_pop = mean + random.normal(key, pop.shape) * sigma
    pop = jnp.clip(new_pop, bounds[:,0], bounds[:,1])
    fit = fitness_fn(pop)
    idx = jnp.argmin(fit)
    best = jnp.where(fit[idx] < best_fit, pop[idx], best)
    best_fit = jnp.minimum(best_fit, fit[idx])
    return pop, fit, best, best_fit

OPTIMIZERS = {
    '1. DE (Differential Evolution)': de_step,
    '2. PSO (Particle Swarm)': pso_step,
    '3. GWO (Grey Wolf)': gwo_step,
    '4. WOA (Whale)': woa_step,
    '5. SCA (Sine-Cosine)': sca_step,
    '6. SSA (Salp Swarm)': ssa_step,
    '7. HHO (Harris Hawks)': hho_step,
    '8. AOA (Arithmetic)': aoa_step,
    '9. CMA-ES': cmaes_step,
}

print(f"✅ {len(OPTIMIZERS)} metaheurísticos definidos:")
for name in OPTIMIZERS: print(f"   • {name}")

In [ ]:
BOUNDS = jnp.array([
    [0.001, 0.020],  # k0 [H]
    [0.001, 0.020],  # k [H]
    [0.002, 0.015],  # a [m]
    [0.005, 0.020],  # m [kg]
    [1.5, 4.0]       # R [Ω]
])
REF = jnp.array([K0_REF, K0_REF, A_REF, M_REF, R_REF])

# Crear función fitness con datos actuales
fitness_fn = partial(fitness_pop, u_data=u_jax, dt_data=dt_jax, y_data=y_jax, i_data=i_jax, 
                     dy_data=dy_jax, ddy_data=ddy_jax, di_data=di_jax, y0=y0, i0=i0, g=GRAVITY, dt=dt_mean)

print("Límites:", [(f"{b[0]*1000:.1f}-{b[1]*1000:.1f}mH" if i<2 else f"{b[0]*1000:.1f}-{b[1]*1000:.1f}mm" if i==2 else f"{b[0]*1000:.0f}-{b[1]*1000:.0f}g" if i==3 else f"{b[0]:.1f}-{b[1]:.1f}Ω") for i,b in enumerate(BOUNDS)])
print(f"\n📊 Optimizadores disponibles: {list(OPTIMIZERS.keys())}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# 🎯 SELECCIÓN DE OPTIMIZADOR - Modificar aquí
# ═══════════════════════════════════════════════════════════
SELECTED_OPTIMIZER = '1. DE (Differential Evolution)'  # Cambiar por cualquier otro
POP_SIZE = 100
MAX_ITER = 150

print(f"🚀 Ejecutando: {SELECTED_OPTIMIZER}")
print(f"   Población: {POP_SIZE}, Iteraciones: {MAX_ITER}")

t0 = time.time()
best_params, best_fit, history = run_optimizer_interactive(
    SELECTED_OPTIMIZER, 
    OPTIMIZERS[SELECTED_OPTIMIZER], 
    fitness_fn, BOUNDS, 
    pop_size=POP_SIZE, max_iter=MAX_ITER, ref=REF)
elapsed = time.time() - t0

print(f"\n⏱️ Tiempo: {elapsed:.1f}s")
print(f"✅ Mejor fitness: {float(best_fit):.6e}")
print(f"   k0={float(best_params[0])*1000:.4f}mH, k={float(best_params[1])*1000:.4f}mH")
print(f"   a={float(best_params[2])*1000:.4f}mm, m={float(best_params[3])*1000:.2f}g, R={float(best_params[4]):.3f}Ω")

## 📊 Comparación de Todos los Optimizadores
Ejecuta todos los metaheurísticos y compara su rendimiento

In [ ]:
# ═══════════════════════════════════════════════════════════
# 🏆 COMPARACIÓN DE TODOS LOS OPTIMIZADORES
# ═══════════════════════════════════════════════════════════
RUN_ALL = False  # Cambiar a True para ejecutar todos

if RUN_ALL:
    results_all = {}
    for name, step_fn in OPTIMIZERS.items():
        print(f"\n{'='*60}")
        print(f"🚀 {name}")
        print('='*60)
        t0 = time.time()
        pso_state = None  # Reset PSO state
        best, fit, hist = run_optimizer_interactive(name, step_fn, fitness_fn, BOUNDS, 
                                                     pop_size=80, max_iter=100, ref=REF)
        results_all[name] = {
            'best_params': np.array(best),
            'best_fit': float(fit),
            'time': time.time() - t0,
            'history': hist['best']
        }
    
    # Tabla comparativa
    print("\n" + "="*80)
    print("📊 RESUMEN COMPARATIVO")
    print("="*80)
    print(f"{'Optimizador':<35} {'Fitness':>12} {'Tiempo':>8} {'k0 [mH]':>10} {'a [mm]':>10}")
    print("-"*80)
    sorted_results = sorted(results_all.items(), key=lambda x: x[1]['best_fit'])
    for name, r in sorted_results:
        print(f"{name:<35} {r['best_fit']:>12.4e} {r['time']:>7.1f}s {r['best_params'][0]*1000:>10.4f} {r['best_params'][2]*1000:>10.4f}")
    
    # Gráfico comparativo
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))
    for name, r in results_all.items():
        ax[0].semilogy(r['history'], label=name.split('.')[1].strip(), lw=1.5)
    ax[0].set_xlabel('Generación'); ax[0].set_ylabel('Fitness'); ax[0].legend(fontsize=8); ax[0].grid(True)
    ax[0].set_title('Convergencia de Optimizadores', fontweight='bold')
    
    names = [n.split('.')[1].strip()[:10] for n in sorted_results]
    fits = [r['best_fit'] for _, r in sorted_results]
    colors = plt.cm.viridis(np.linspace(0, 1, len(names)))
    ax[1].barh(names, fits, color=colors)
    ax[1].set_xlabel('Fitness (menor es mejor)'); ax[1].set_xscale('log')
    ax[1].set_title('Ranking de Optimizadores', fontweight='bold')
    plt.tight_layout(); plt.savefig('optimizer_comparison.png', dpi=150); plt.show()
else:
    print("ℹ️ Cambia RUN_ALL = True para ejecutar todos los optimizadores")

In [ ]:
# Visualización de resultados
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Convergence
axes[0,0].semilogy(history['best'], 'b-', lw=2, label='Best Fitness')
axes[0,0].set_xlabel('Generación'); axes[0,0].set_ylabel('Fitness'); axes[0,0].legend(); axes[0,0].grid(True)
axes[0,0].set_title(f'Convergencia - {SELECTED_OPTIMIZER}', fontweight='bold')

# Parameter bar chart
params_labels = ['k0 [mH]', 'k [mH]', 'a [mm]', 'm [g]', 'R [Ω]']
params_values = [float(best_params[0])*1000, float(best_params[1])*1000, float(best_params[2])*1000, 
                 float(best_params[3])*1000, float(best_params[4])]
ref_values = [K0_REF*1000, K0_REF*1000, A_REF*1000, M_REF*1000, R_REF]
x = np.arange(len(params_labels))
axes[0,1].bar(x - 0.2, params_values, 0.4, label='Identificado', color='steelblue')
axes[0,1].bar(x + 0.2, ref_values, 0.4, label='Referencia', color='coral', alpha=0.7)
axes[0,1].set_xticks(x); axes[0,1].set_xticklabels(params_labels)
axes[0,1].legend(); axes[0,1].grid(True, axis='y')
axes[0,1].set_title('Parámetros Identificados vs Referencia', fontweight='bold')

# Position comparison
y_opt, i_opt, _ = simulate_trajectory(best_params, u_jax, dt_jax, y0, i0, GRAVITY)
axes[1,0].plot(data['t'], data['y']*1000, 'b-', lw=1.5, label='Experimental', alpha=0.7)
axes[1,0].plot(data['t'], np.array(y_opt)*1000, 'r--', lw=1.5, label='Simulado')
axes[1,0].set_xlabel('t [s]'); axes[1,0].set_ylabel('y [mm]'); axes[1,0].legend(); axes[1,0].grid(True)
axes[1,0].set_title('Comparación de Posición', fontweight='bold')

# Current comparison
axes[1,1].plot(data['t'], data['i'], 'b-', lw=1.5, label='Experimental', alpha=0.7)
axes[1,1].plot(data['t'], np.array(i_opt), 'r--', lw=1.5, label='Simulado')
axes[1,1].set_xlabel('t [s]'); axes[1,1].set_ylabel('i [A]'); axes[1,1].legend(); axes[1,1].grid(True)
axes[1,1].set_title('Comparación de Corriente', fontweight='bold')

plt.tight_layout(); plt.savefig('results.png', dpi=150); plt.show()

# Calcular métricas
mse_y = np.mean((data['y'] - np.array(y_opt))**2)
mse_i = np.mean((data['i'] - np.array(i_opt))**2)
print(f"\n📐 Métricas de ajuste:")
print(f"   MSE posición: {mse_y:.6e} m²")
print(f"   MSE corriente: {mse_i:.6e} A²")
print(f"   RMSE posición: {np.sqrt(mse_y)*1000:.4f} mm")
print(f"   RMSE corriente: {np.sqrt(mse_i):.4f} A")

In [ ]:
# Visualización 3D interactiva del resultado final
fig_3d = go.Figure()

# Punto óptimo encontrado
fig_3d.add_trace(go.Scatter3d(
    x=[float(best_params[0])*1000], y=[float(best_params[1])*1000], z=[float(best_params[2])*1000],
    mode='markers+text', marker=dict(size=15, color='lime', symbol='diamond'),
    text=['Óptimo'], textposition='top center', name='Parámetros Identificados'))

# Punto de referencia
fig_3d.add_trace(go.Scatter3d(
    x=[K0_REF*1000], y=[K0_REF*1000], z=[A_REF*1000],
    mode='markers+text', marker=dict(size=12, color='red', symbol='cross'),
    text=['Referencia'], textposition='bottom center', name='Referencia Santana'))

# Límites del espacio de búsqueda
fig_3d.update_layout(
    template="plotly_dark",
    title=f"Resultado Final: k0={float(best_params[0])*1000:.3f}mH, k={float(best_params[1])*1000:.3f}mH, a={float(best_params[2])*1000:.3f}mm",
    scene=dict(
        xaxis=dict(title='k0 [mH]', range=[BOUNDS[0,0]*1000, BOUNDS[0,1]*1000]),
        yaxis=dict(title='k [mH]', range=[BOUNDS[1,0]*1000, BOUNDS[1,1]*1000]),
        zaxis=dict(title='a [mm]', range=[BOUNDS[2,0]*1000, BOUNDS[2,1]*1000])),
    width=800, height=600)
fig_3d.show()

print("✅ Visualización 3D interactiva generada")

In [ ]:
print("="*60)
print("📊 RESULTADOS FINALES - PRINCIPIO DE MÍNIMA ACCIÓN")
print("="*60)
print(f"\n🎯 Parámetros Identificados:")
print(f"   k0 = {float(best_params[0])*1000:.4f} mH  (Inductancia base)")
print(f"   k  = {float(best_params[1])*1000:.4f} mH  (Coef. inductancia)")
print(f"   a  = {float(best_params[2])*1000:.4f} mm  (Parámetro geométrico)")
print(f"   m  = {float(best_params[3])*1000:.4f} g   (Masa)")
print(f"   R  = {float(best_params[4]):.4f} Ω    (Resistencia)")

print(f"\n📐 Comparación con Referencia (Santana 2023):")
print(f"   k0: {float(best_params[0])*1000:.4f} vs {K0_REF*1000:.4f} mH (Δ={abs(float(best_params[0])-K0_REF)*1000:.4f})")
print(f"   a:  {float(best_params[2])*1000:.4f} vs {A_REF*1000:.4f} mm (Δ={abs(float(best_params[2])-A_REF)*1000:.4f})")
print(f"   m:  {float(best_params[3])*1000:.4f} vs {M_REF*1000:.4f} g  (Δ={abs(float(best_params[3])-M_REF)*1000:.4f})")
print(f"   R:  {float(best_params[4]):.4f} vs {R_REF:.4f} Ω  (Δ={abs(float(best_params[4])-R_REF):.4f})")

print(f"\n✅ Fitness final: {float(best_fit):.6e}")

# Save results
results = {
    'params': {'k0': float(best_params[0]), 'k': float(best_params[1]), 'a': float(best_params[2]), 
               'm': float(best_params[3]), 'R': float(best_params[4])},
    'fitness': float(best_fit),
    'reference': {'k0': K0_REF, 'a': A_REF, 'm': M_REF, 'R': R_REF},
    'timestamp': datetime.now().isoformat()
}
with open('lagrangian_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print(f"\n💾 Resultados guardados en 'lagrangian_results.json'")